In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import emoji
from string import punctuation

In [2]:
def remove_punc(string):
    punctuation_zh = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~“”？，！【】（）、。：；’‘……￥·"""
    dicts = {i: '' for i in punctuation + punctuation_zh}
    dicts.pop('#')  # keep#
    punc_table = str.maketrans(dicts)
    return string.translate(punc_table)

In [3]:
def remove_emoji(string):
    allchars = [str for str in string]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI['en']]
    clean_text = ' '.join([str for str in string.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [4]:
def read_data(file):
    texts = []
    labels = []
    data = pd.read_csv(file)
    data.dropna(subset=['tidyTweet', 'Label'], inplace=True)
    for i in tqdm(data.itertuples(), total=len(data)):
        line = getattr(i, 'tidyTweet')
        line = line.strip()
        if not line:
            continue
        line = remove_emoji(line)
        line = remove_punc(line).strip()
        texts.append(line)
        label = getattr(i, 'Label')
        labels.append(label)
        if label == 1.0:
            texts.extend([line] * 2)
            labels.extend([label] * 2)
    assert len(texts) == len(labels)
    return texts, labels

In [5]:
texts, labels = read_data("./bert/733 label data - 2.csv")

  0%|          | 0/12058 [00:00<?, ?it/s]

In [16]:
df = pd.DataFrame({'text':texts, 'label':labels})
print(len(df))
df = df[~df['text'].str.contains('#depop ')].reset_index(drop=True)
print(len(df))

17820
17549


In [17]:
df.to_csv('./dataset.csv')